In [20]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import itertools
import random

from dask.distributed import Client, progress
import dask
import dask.dataframe as dd
client.shutdown()
client.close()
dask.config.set({'temporary_directory': '/pasteur/sonic/scratch/public/cduitama/RascovanProject/tmp/'})
#dask.config.set({"optimization.fuse.ave-width": 5})
client = Client()
client

distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing


Client Scheduler: tcp://127.0.0.1:36287 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 18 Memory: 1.76 TiB


# Classification

## Data preparation

In [8]:
#Removal of stool samples
SRR_aSGenomes_HS_Oral=pd.read_csv("Metadata/aSGenomes/aSGenomes_HS_Oral_SRR.txt",header=None,names=["SRR_accession"])
SRR_mMetagenomes_HS_Oral=pd.read_csv("Metadata/mMetagenomes/Oral_HS_mMetagenomes_SRR.txt",\
                                     header=None,names=["SRR_accession"])

HS_Oral_metadata=pd.read_csv("Metadata/aMetagenomes/HS_Oral_metadata.csv")
SRR_HS_Oral=["Kmer"]+list(HS_Oral_metadata["SRR_accession"])


SRR_HS_Oral_aMetagenomes=SRR_HS_Oral[1:]
SRR_HS_Oral_aSGenomes=list(SRR_aSGenomes_HS_Oral.SRR_accession)
SRR_HS_Oral_mMetagenomes=list(SRR_mMetagenomes_HS_Oral.SRR_accession)



names=SRR_HS_Oral+SRR_HS_Oral_aSGenomes+SRR_HS_Oral_mMetagenomes
combined_total=dd.read_table("kmMatrices/combination/combination_whole_matrix.txt",sep=" ",header=None,names=names)

import csv

with open('Metadata/combination/stool_accessions.txt', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

stool_accessions=data[0]

combined_total_clean=combined_total.drop(stool_accessions,axis=1)
clean_names=[x for x in names if x not in stool_accessions]


### Labelling

In [4]:
to_take_2=np.genfromtxt("Classification/list_of_indeces_of_CleanRepresentativeSample.csv",delimiter=",")

In [5]:
len(to_take_2)

9530199

In [27]:
classification_2=pd.read_csv("Preprocessed_data/combination/classification_2.csv")

In [112]:
classification_2=pd.read_csv("Preprocessed_data/combination/classification_2.csv")
mMetagenomes_2=set(classification_2[(classification_2.Perc_mMetagenome > float(0))].index.values.tolist())
aMetagenomes_2=set(classification_2[(classification_2.Perc_aMetagenome > float(0))].index.values.tolist())
aSGenomes_2=set(classification_2[(classification_2.Perc_aSGenome > float(0))].index.values.tolist())
intersection_2=set.intersection(aMetagenomes_2,mMetagenomes_2,aSGenomes_2)
aMetagenomes_aSGenomes_2=aMetagenomes_2.intersection(aSGenomes_2)
aMetagenomes_mMetagenomes_2=aMetagenomes_2.intersection(mMetagenomes_2)
mMetagenomes_aSGenomes_2=mMetagenomes_2.intersection(aSGenomes_2)
onlymMetagenomes_2=mMetagenomes_2-mMetagenomes_aSGenomes_2-aMetagenomes_mMetagenomes_2
onlyaSGenomes_2=aSGenomes_2-mMetagenomes_aSGenomes_2-aMetagenomes_aSGenomes_2
onlyaMetagenomes_2=aMetagenomes_2-aMetagenomes_aSGenomes_2-aMetagenomes_mMetagenomes_2

one=intersection_2.intersection(set(to_take_2))
two=onlyaMetagenomes_2.intersection(set(to_take_2))
three=onlyaSGenomes_2.intersection(set(to_take_2))
four=onlymMetagenomes_2.intersection(set(to_take_2))
five=set(aMetagenomes_aSGenomes_2-intersection_2).intersection(set(to_take_2))
six=set(aMetagenomes_mMetagenomes_2-intersection_2).intersection(set(to_take_2))
seven=set(mMetagenomes_aSGenomes_2-intersection_2).intersection(set(to_take_2))

In [ ]:
#1=Ancient
#2=NonAncient
to_take=list(one)+list(two)+list(three)+list(four)+list(five)+list(six)+list(seven)
labels=[]
labels=labels+[False]*len(one)
labels=labels+[False]*len(two)
labels=labels+[True]*len(three)
labels=labels+[False]*len(four)
labels=labels+[True]*len(five)
labels=labels+[False]*len(six)
labels=labels+[True]*len(seven)
Data = pd.DataFrame(list(zip(to_take, labels)),
               columns =['Index', 'Label'])
Data.sort_values(by='Index' , inplace=True)
Data=Data.reset_index()

Data=Data.drop("index",axis=1)
Data.head()

np.savetxt("Classification/Labels_CleanRepresentativeSample.csv",Data["Label"], delimiter=',')



In [5]:
labels_CleanRepresentativeSample=np.genfromtxt("Classification/Labels_CleanRepresentativeSample.csv",delimiter=",")

## Data splitting

In [6]:
labels_CleanRepresentativeSample.shape

(9530199,)

In [7]:
ikmers=dict()
imatrices=dict()

all_kmers=combined_total_clean.Kmer.compute()
all_kmers=all_kmers.reset_index()
all_kmers=all_kmers.drop("index",axis=1)
ikmers["CleanRepresentativeSample"]=all_kmers.iloc[to_take_2]

icombined_total_clean=combined_total_clean[combined_total_clean["Kmer"].isin(ikmers["CleanRepresentativeSample"]["Kmer"])]

imatrices["CleanRepresentativeSample"]=icombined_total_clean.drop("Kmer",axis=1).values

In [8]:
from sklearn.preprocessing import Binarizer
icombined_total=imatrices["CleanRepresentativeSample"]
transformer = Binarizer().fit(icombined_total)  # fit does nothing.
b_icombined_total=transformer.transform(icombined_total)

/pasteur/sonic/scratch/public/cduitama/Software/anaconda3/lib/python3.8/site-packages/dask/array/core.py:1488: FutureWarning: The `numpy.may_share_memory` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(


In [126]:
#Split data into training and testing with non-blockwise shuffling
import dask.array as da
from dask_ml.model_selection import train_test_split
X=b_icombined_total
y=labels_CleanRepresentativeSample
#y=y.rechunk((55555, 1))
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.5,shuffle=True,blockwise=False)

In [10]:
print(X_train.shape)

print(y_train.shape)

print(X_test.shape)

print(y_test.shape)

(4765099, 294)
(4765099,)
(4765100, 294)
(4765100,)


# Logistic regression

In [171]:
import joblib
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(class_weight="balanced",n_jobs=-1)
with joblib.parallel_backend('dask'):
    LR.fit(X_train, y_train)
    y_test_predicted_LR=LR.predict(X_test)
    y_train_predicted_LR=LR.predict(X_train)
    y_train_score_LR=LR.predict_proba(X_train)[:, 1]
    y_test_score_LR=LR.predict_proba(X_test)[:, 1]

In [14]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_train, y_train_predicted_LR)


0.9984347773141473

In [15]:
balanced_accuracy_score(y_test, y_test_predicted_LR)

0.9983589784437881

In [172]:
np.savetxt("Classification/LR/y_test_predicted_labels.csv",y_test_predicted_LR,delimiter=",")
np.savetxt("Classification/LR/y_train_predicted_labels.csv",y_train_predicted_LR,delimiter=",")
np.savetxt("Classification/LR/y_test_true_labels.csv",y_test,delimiter=",")
np.savetxt("Classification/LR/y_train_true_labels.csv",y_train,delimiter=",")
np.savetxt("Classification/LR/y_train_score_LR.csv",y_train_score_LR,delimiter=",")
np.savetxt("Classification/LR/y_test_score_LR.csv",y_test_score_LR,delimiter=",")
np.savetxt("Classification/LR/Coefficients.csv",LR.coef_[0],delimiter=",")

In [20]:
y_test_predicted_LR=np.genfromtxt("Classification/LR/y_test_predicted_labels.csv",delimiter=",")
y_train_predicted_LR=np.genfromtxt("Classification/LR/y_train_predicted_labels.csv",delimiter=",")
y_test=np.genfromtxt("Classification/LR/y_test_true_labels.csv",delimiter=",")
y_train=np.genfromtxt("Classification/LR/y_train_true_labels.csv",delimiter=",")
y_train_score_LR=np.genfromtxt("Classification/LR/y_train_score_LR.csv",delimiter=",")
y_test_score_LR=np.genfromtxt("Classification/LR/y_test_score_LR.csv",delimiter=",")
LR_coef=np.genfromtxt("Classification/LR/Coefficients.csv",delimiter=",")

In [173]:
combination_metadata=pd.read_csv("Metadata/combination/combination_metadata.csv")
import csv
with open('Metadata/combination/stool_accessions.txt', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

stool_accessions=data[0]
combination_metadata=combination_metadata[-combination_metadata["Run_accession"].isin(stool_accessions)]

In [174]:
to_merge=combination_metadata[["Run_accession","Type"]]

In [175]:
Coef=pd.DataFrame()
Coef["Coefficients"]=np.absolute(LR_coef)
Coef["Run_accession"]=clean_names[1:]
Coef=Coef.merge(to_merge,on="Run_accession")
fig = px.bar(Coef, x='Run_accession', y='Coefficients',hover_data=['Type'],color='Type')
fig.write_html("Classification/LR/Plot_Regressors.html")

In [196]:
Coef.to_csv("Classification/LR/Coefficients_DF.csv",index=False)

In [176]:
fig.show()

# Random Forest

In [177]:
#Split data into training and testing with non-blockwise shuffling
import dask.array as da
from dask_ml.model_selection import train_test_split
X=b_icombined_total
y=labels_CleanRepresentativeSample
X_train_small, X_test_big, y_train_small, y_test_big = train_test_split(X, y,train_size=0.01,shuffle=True,blockwise=False)

In [178]:
print(X_train_small.shape)

print(y_train_small.shape)

print(X_test_big.shape)

print(y_test_big.shape)

(95301, 294)
(95301,)
(9434898, 294)
(9434898,)


In [179]:
from sklearn.ensemble import RandomForestClassifier
import joblib
RF = RandomForestClassifier(class_weight="balanced",n_jobs=-1)
with joblib.parallel_backend('dask'):
    RF.fit(X_train_small, y_train_small)
    y_test_predicted_RF=LR.predict(X_test_big)
    y_train_predicted_RF=LR.predict(X_train_small)
    y_train_score_RF=RF.predict_proba(X_train_small)[:, 1]
    y_test_score_RF=RF.predict_proba(X_test_big)[:, 1]


In [47]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test_big, y_test_predicted_RF)

0.998394409626005

In [180]:
np.savetxt("Classification/RF/y_test_predicted_labels.csv",y_test_predicted_RF,delimiter=",")
np.savetxt("Classification/RF/y_train_predicted_labels.csv",y_train_predicted_RF,delimiter=",")
np.savetxt("Classification/RF/y_test_true_labels.csv",y_test_big,delimiter=",")
np.savetxt("Classification/RF/y_train_true_labels.csv",y_train_small,delimiter=",")
np.savetxt("Classification/RF/y_train_score_RF.csv",y_train_score_RF,delimiter=",")
np.savetxt("Classification/RF/y_test_score_RF.csv",y_test_score_RF,delimiter=",")

# SVM

ELastic net penalization + SGD training + SVM  

Weights are adjusted in an inversely proportional way to class frequencies of the input data as n_samples

In [181]:
from sklearn.linear_model import SGDClassifier
import joblib
SVM = SGDClassifier(class_weight="balanced",n_jobs=-1)
with joblib.parallel_backend('dask'):
    SVM.fit(X_train, y_train)
    y_test_predicted_SVM=SVM.predict(X_test)
    y_train_predicted_SVM=SVM.predict(X_train)
    y_train_score_SVM=SVM.decision_function(X_train)
    y_test_score_SVM=SVM.decision_function(X_test)

In [182]:
np.savetxt("Classification/SVM/y_test_predicted_labels.csv",y_test_predicted_SVM,delimiter=",")
np.savetxt("Classification/SVM/y_train_predicted_labels.csv",y_train_predicted_SVM,delimiter=",")
np.savetxt("Classification/SVM/y_test_true_labels.csv",y_test,delimiter=",")
np.savetxt("Classification/SVM/y_train_true_labels.csv",y_train,delimiter=",")
np.savetxt("Classification/SVM/y_train_score_SVM.csv",y_train_score_SVM,delimiter=",")
np.savetxt("Classification/SVM/y_test_score_SVM.csv",y_test_score_SVM,delimiter=",")

In [80]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_train,y_train_predicted_LR)

0.9984347773141473

In [81]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,y_train_predicted_LR)

0.997841807693817

# Balanced Bagging + Histogram Gradient Boosting

In [183]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
BB=BalancedBaggingClassifier(n_jobs=-1,base_estimator=HistGradientBoostingClassifier())
with joblib.parallel_backend('dask'):
    BB.fit(X_train, y_train)
    y_test_predicted_BB=BB.predict(X_test)
    y_train_predicted_BB=BB.predict(X_train)
    y_train_score_BB=BB.decision_function(X_train)
    y_test_score_BB=BB.decision_function(X_test)

In [184]:
np.savetxt("Classification/Boosting+Bagging/y_test_predicted_labels.csv",y_test_predicted_BB,delimiter=",")
np.savetxt("Classification/Boosting+Bagging/y_train_predicted_labels.csv",y_train_predicted_BB,delimiter=",")
np.savetxt("Classification/Boosting+Bagging/y_test_true_labels.csv",y_test,delimiter=",")
np.savetxt("Classification/Boosting+Bagging/y_train_true_labels.csv",y_train,delimiter=",")
np.savetxt("Classification/Boosting+Bagging/y_train_score_BB.csv",y_train_score_BB,delimiter=",")
np.savetxt("Classification/Boosting+Bagging/y_test_score_BB.csv",y_test_score_BB,delimiter=",")

In [85]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,y_train_predicted_BB)

0.9996843717202938

In [185]:
ModelComparison=pd.DataFrame()
ModelComparison["Models"]=["Logistic Regression","Random Forest Classifier","SVM", "Histogram Gradient Boosting"]
ModelComparison["Training set/Dataset size ratio"]=[np.round(X_train.shape[0]/X.shape[0],2),\
                                                   np.round(X_train_small.shape[0]/X.shape[0],2),\
                                                    np.round(X_train.shape[0]/X.shape[0],2),\
                                                    np.round(X_train.shape[0]/X.shape[0],2)]
ModelComparison["Test set/Dataset size ratio"]=[np.round(X_test.shape[0]/X.shape[0],2),\
                                                   np.round(X_test_big.shape[0]/X.shape[0],2),\
                                                    np.round(X_test.shape[0]/X.shape[0],2),\
                                                    np.round(X_test.shape[0]/X.shape[0],2)]

In [186]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

ModelComparison["Imbalance correction"]=["Weights inversely proportional to class frequencies",\
                                        "Weights inversely proportional to class frequencies",\
                                        "Weigths inversely proportional to class frequencies",\
                                        "Under sampling before every boosting sample"]
ModelComparison["Acccuracy Training"]=[np.round(accuracy_score(y_train,y_train_predicted_LR),6),\
                                      np.round(accuracy_score(y_train_small,y_train_predicted_RF),6),\
                                      np.round(accuracy_score(y_train,y_train_predicted_SVM),6),\
                                      np.round(accuracy_score(y_train,y_train_predicted_BB),6),]
ModelComparison["Acccuracy Testing"]=[np.round(accuracy_score(y_test,y_test_predicted_LR),6),\
                                      np.round(accuracy_score(y_test_big,y_test_predicted_RF),6),\
                                      np.round(accuracy_score(y_test,y_test_predicted_SVM),6),\
                                      np.round(accuracy_score(y_test,y_test_predicted_BB),6),]
ModelComparison["Balanced Acccuracy Training"]=[np.round(balanced_accuracy_score(y_train,y_train_predicted_LR),6),\
                                      np.round(balanced_accuracy_score(y_train_small,y_train_predicted_RF),6),\
                                      np.round(balanced_accuracy_score(y_train,y_train_predicted_SVM),6),\
                                      np.round(balanced_accuracy_score(y_train,y_train_predicted_BB),6)]
ModelComparison["Balanced Acccuracy Testing"]=[np.round(balanced_accuracy_score(y_test,y_test_predicted_LR),6),\
                                      np.round(balanced_accuracy_score(y_test_big,y_test_predicted_RF),6),\
                                      np.round(balanced_accuracy_score(y_test,y_test_predicted_SVM),6),\
                                      np.round(balanced_accuracy_score(y_test,y_test_predicted_BB),6)]
ModelComparison["Precision Training"]=[np.round(precision_score(y_train,y_train_predicted_LR),6),\
                                      np.round(precision_score(y_train_small,y_train_predicted_RF),6),\
                                      np.round(precision_score(y_train,y_train_predicted_SVM),6),\
                                      np.round(precision_score(y_train,y_train_predicted_BB),6)]
ModelComparison["Precision Testing"]=[np.round(precision_score(y_test,y_test_predicted_LR),6),\
                                      np.round(precision_score(y_test_big,y_test_predicted_RF),6),\
                                      np.round(precision_score(y_test,y_test_predicted_SVM),6),\
                                      np.round(precision_score(y_test,y_test_predicted_BB),6)]
ModelComparison["Recall Training"]=[np.round(precision_score(y_train,y_train_predicted_LR),6),\
                                      np.round(precision_score(y_train_small,y_train_predicted_RF),6),\
                                      np.round(precision_score(y_train,y_train_predicted_SVM),6),\
                                      np.round(precision_score(y_train,y_train_predicted_BB),6)]
ModelComparison["Recall Testing"]=[np.round(precision_score(y_test,y_test_predicted_LR),6),\
                                      np.round(precision_score(y_test_big,y_test_predicted_RF),6),\
                                      np.round(precision_score(y_test,y_test_predicted_SVM),6),\
                                      np.round(precision_score(y_test,y_test_predicted_BB),6)]
ModelComparison["Recall Training"]=[np.round(recall_score(y_train,y_train_predicted_LR),6),\
                                      np.round(recall_score(y_train_small,y_train_predicted_RF),6),\
                                      np.round(recall_score(y_train,y_train_predicted_SVM),6),\
                                      np.round(recall_score(y_train,y_train_predicted_BB),6)]
ModelComparison["Recall Testing"]=[np.round(recall_score(y_test,y_test_predicted_LR),6),\
                                      np.round(recall_score(y_test_big,y_test_predicted_RF),6),\
                                      np.round(recall_score(y_test,y_test_predicted_SVM),6),\
                                      np.round(recall_score(y_test,y_test_predicted_BB),6)]
ModelComparison["Recall Training"]=[np.round(recall_score(y_train,y_train_predicted_LR),6),\
                                      np.round(recall_score(y_train_small,y_train_predicted_RF),6),\
                                      np.round(recall_score(y_train,y_train_predicted_SVM),6),\
                                      np.round(recall_score(y_train,y_train_predicted_BB),6)]
ModelComparison["Recall Testing"]=[np.round(recall_score(y_test,y_test_predicted_LR),6),\
                                      np.round(recall_score(y_test_big,y_test_predicted_RF),6),\
                                      np.round(recall_score(y_test,y_test_predicted_SVM),6),\
                                      np.round(recall_score(y_test,y_test_predicted_BB),6)]
ModelComparison["AUC Training"]=[np.round(roc_auc_score(y_train,y_train_predicted_LR),6),\
                                      np.round(roc_auc_score(y_train_small,y_train_predicted_RF),6),\
                                      np.round(roc_auc_score(y_train,y_train_predicted_SVM),6),\
                                      np.round(roc_auc_score(y_train,y_train_predicted_BB),6)]
ModelComparison["AUC Testing"]=[np.round(roc_auc_score(y_test,y_test_predicted_LR),6),\
                                      np.round(roc_auc_score(y_test_big,y_test_predicted_RF),6),\
                                      np.round(roc_auc_score(y_test,y_test_predicted_SVM),6),\
                                      np.round(roc_auc_score(y_test,y_test_predicted_BB),6)]

In [187]:
ModelComparison

,Models,Training set/Dataset size ratio,Test set/Dataset size ratio,Imbalance correction,Acccuracy Training,Acccuracy Testing,Balanced Acccuracy Training,Balanced Acccuracy Testing,Precision Training,Precision Testing,Recall Training,Recall Testing,AUC Training,AUC Testing
0,Logistic Regression,0.50,0.50,Weights inversely proportional to class freque...,0.997849,0.997793,0.998448,0.998299,0.959548,0.959021,0.999115,0.998863,0.998448,0.998299
1,Random Forest Classifier,0.01,0.99,Weights inversely proportional to class freque...,0.997660,0.997822,0.998059,0.998377,0.955738,0.959318,0.998501,0.998993,0.998059,0.998377
2,SVM,0.50,0.50,Weigths inversely proportional to class freque...,0.995522,0.995452,0.997643,0.997605,0.917844,0.917190,1.000000,1.000000,0.997643,0.997605
3,Histogram Gradient Boosting,0.50,0.50,Under sampling before every boosting sample,0.999690,0.999680,0.999827,0.999804,0.993871,0.993748,0.999979,0.999942,0.999827,0.999804


In [188]:
ModelComparison.to_csv("Classification/ModelComparison_metrics.csv",header=True,index=False,float_format="%.6f")

# Validation

In [114]:
to_take_v=[]
n=100000
one_v=random.sample(intersection_2-set(to_take_2),int(0.456*n))
two_v=random.sample(onlyaMetagenomes_2-set(to_take_2),int(59.275*n))
three_v=random.sample(onlyaSGenomes_2-set(to_take_2),int(0.001*n))
four_v=random.sample(onlymMetagenomes_2-set(to_take_2), int(16.103*n))
five_v=random.sample(aMetagenomes_aSGenomes_2-intersection_2-set(to_take_2),int(4.745*n))
six_v=random.sample(aMetagenomes_mMetagenomes_2-intersection_2-set(to_take_2),int(19.393*n))
seven_v=random.sample(mMetagenomes_aSGenomes_2-intersection_2-set(to_take_2),int(0.026*n))
to_take_v=one_v+two_v+three_v+four_v+five_v+six_v+seven_v
to_take_v.sort()

In [119]:
ikmers["RepresentativeSample_Validation"]=all_kmers.iloc[to_take_v]
icombined_total_clean=combined_total_clean[combined_total_clean["Kmer"].isin(ikmers["RepresentativeSample_Validation"]["Kmer"])]
imatrices["RepresentativeSample_Validation"]=icombined_total_clean.drop("Kmer",axis=1).values
imatrices["RepresentativeSamplen_Validation"]=imatrices["RepresentativeSample_Validation"].persist()

from sklearn.preprocessing import Binarizer
ecombined_total_v=imatrices["RepresentativeSample_Validation"]
b_ecombined_total_v = Binarizer().fit_transform(ecombined_total_v)  # fit does nothing.


distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/pasteur/sonic/scratch/public/cduitama/Software/anaconda3/lib/python3.8/site-packages/distributed/core.py", line 497, in handle_comm
    result = handler(comm, **msg)
  File "/pasteur/sonic/scratch/public/cduitama/Software/anaconda3/lib/python3.8/site-packages/distributed/scheduler.py", line 3647, in heartbeat_worker
    ws._executing = {
  File "/pasteur/sonic/scratch/public/cduitama/Software/anaconda3/lib/python3.8/site-packages/distributed/scheduler.py", line 3648, in <dictcomp>
    parent._tasks[key]: duration for key, duration in executing.items()
KeyError: "('read-csv-values-values-b6aeabee9716e82c1a1fc66314dedbdc', 6772, 0)"
distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/pasteur/sonic/scratch/public/cduitama/Software/anaconda3/lib/python3.8/site-packages/distributed/core.py", line 497, in handle_co

In [120]:
b_ecombined_total_v.shape

(9999900, 294)

In [121]:
#1=Ancient
#2=NonAncient
to_take_v=one_v+two_v+three_v+four_v+five_v+six_v+seven_v
labels=[]
labels=labels+[False]*len(one_v)
labels=labels+[False]*len(two_v)
labels=labels+[True]*len(three_v)
labels=labels+[False]*len(four_v)
labels=labels+[True]*len(five_v)
labels=labels+[False]*len(six_v)
labels=labels+[True]*len(seven_v)
Data = pd.DataFrame(list(zip(to_take_v, labels)),
               columns =['Index', 'Label'])
Data.sort_values(by='Index' , inplace=True)
Data=Data.reset_index()

Data=Data.drop("index",axis=1)
Data.head()

np.savetxt("Classification/Labels_CleanRepresentativeSample_Validation.csv",Data["Label"], delimiter=',')

labels_RepresentativeSample_Validation=np.genfromtxt("Classification/Labels_CleanRepresentativeSample_Validation.csv",delimiter=",")

X_v=b_ecombined_total_v
y_v=labels_RepresentativeSample_Validation

In [122]:
np.savetxt("Classification/list_of_indeces_of_CleanRepresentativeSample_Validation.csv",to_take_v,delimiter=",")

to_take_v=np.genfromtxt("Classification/list_of_indeces_of_CleanRepresentativeSample_Validation.csv",delimiter=",")

len(to_take_v)

9999900

In [189]:
y_validation_predicted_LR=LR.predict(b_ecombined_total_v)

In [190]:
np.savetxt("Classification/LR/Clean_y_validation_predicted_LR.csv",y_validation_predicted_LR,delimiter=",")

In [191]:
pd.set_option("display.precision", 6)
ValidationMetrics=pd.DataFrame()
ValidationMetrics["Metrics"]=["Accuracy","Balanced accuracy","Precision","Recall","AUC"]
ValidationMetrics["Value"]=[np.round(accuracy_score(y_v,y_validation_predicted_LR),6),\
                           np.round(balanced_accuracy_score(y_v,y_validation_predicted_LR),6),\
                           np.round(precision_score(y_v,y_validation_predicted_LR),6),\
                           np.round(recall_score(y_v,y_validation_predicted_LR),6),\
                           np.round(roc_auc_score(y_v,y_validation_predicted_LR),6)]

In [192]:
ValidationMetrics.to_csv("Classification/LR/ValidationMetrics.csv",index=False,float_format="%.6f")

In [193]:
y_v_score = LR.predict_proba(X_v)[:, 1]

In [194]:
np.savetxt("Classification/LR/Clean_y_validation_score.csv",y_v_score,delimiter=",")

In [145]:
from sklearn.metrics import recall_score
recall_score(y_v,y_validation_predicted_LR)

0.9989291701592624

In [195]:
ValidationMetrics

,Metrics,Value
0,Accuracy,0.997879
1,Balanced accuracy,0.998375
2,Precision,0.958388
3,Recall,0.998923
4,AUC,0.998375


In [197]:
import joblib

joblib.dump(LR,"Classification/LR/Model.sav")
joblib.dump(RF,"Classification/RF/Model.sav")
joblib.dump(SVM,"Classification/SVM/Model.sav")
joblib.dump(BB,"Classification/Boosting+Bagging/Model.sav")

 


['Classification/Boosting+Bagging/Model.sav']

# Experiment for feature importance

In [4]:
clean_SRR_HS_Oral_mMetagenomes=[x for x in SRR_HS_Oral_mMetagenomes if x not in stool_accessions]

In [ ]:
#Dictionaries:

#labelling_features = Features selected for labelling only
#           keys: Split number


#modelling_features = Features selected for modelling only
#           keys: Split number

#datasets = The idea of this dictionary is to store the SRR/ERR accessions classified per subkey to build the
#           set of features used for labelling
#           keys: Split number
#           subkeys: aMetagenomes,aSGenomes,mMetagenomes

#labelling_matrices = This dictionary stores the matrices used for labelling 
#           keys: Split number

#modelling_matrices = This dictionary stores the matrices used for modelling 
#           keys: Split number
          
#indeces = This dictionary stores the classification of k-mers based on the Venn Diagram obtained per labelling matrix
#           keys: Split number
#           subkeys: ["intersection","aMetagenomes_aSGenomes","aMetagenomes_mMetagenomes",\
#                     "mMetagenomes_aSGenomes","onlyaMetagenomes","onlymMetagenomes","onlyaSGenomes"]

#modelling features = This dictionary stores the classification of k-mers based on the Venn Diagram obtained per labelling matrix
#           keys: Split number
#           subkeys: ["intersection","aMetagenomes_aSGenomes","aMetagenomes_mMetagenomes",\
#                     "mMetagenomes_aSGenomes","onlyaMetagenomes","onlymMetagenomes","onlyaSGenomes"]

#proportions = This dictionary stores the proportion of k-mers on each group of the Venn Diagram obtained per labelling matrix
#           keys: Split number
#           subkeys: ["intersection","aMetagenomes_aSGenomes","aMetagenomes_mMetagenomes",\
#                     "mMetagenomes_aSGenomes","onlyaMetagenomes","onlymMetagenomes","onlyaSGenomes"]


#modelling_samples = This dictionary stores the indeces of the k-mers used for training and testing (modelling) 
#           keys: Split number

#modelling_samples = This dictionary stores the indeces of the k-mers used for training and testing (modelling) 
#           keys: Split number

#labels_per_dataset = This dictionary stores the labels of the kmers stores in modelling_samples
#           keys: Split number

#binarized_modelling_matrices = This dictionary stores the matrices used for modelling

          
import random
labelling_features=dict()
datasets=dict.fromkeys(range(0,10))
labelling_matrices=dict.fromkeys(range(0,10))
modelling_matrices=dict.fromkeys(range(0,10))
indeces=dict.fromkeys(range(0,10))
modelling_features=dict()

for each in datasets.keys():
    datasets[each]=dict.fromkeys(["aMetagenomes","aSGenomes","mMetagenomes"])
    indeces[each]=dict.fromkeys(["intersection","aMetagenomes_aSGenomes","aMetagenomes_mMetagenomes",\
                                 "mMetagenomes_aSGenomes","onlyaMetagenomes","onlymMetagenomes","onlyaSGenomes"])
for i in range(0,10):
    datasets[i]["aMetagenomes"]=random.sample(SRR_HS_Oral_aMetagenomes,140)
    datasets[i]["aSGenomes"]=random.sample(SRR_HS_Oral_aSGenomes,2)
    datasets[i]["mMetagenomes"]=random.sample(clean_SRR_HS_Oral_mMetagenomes,4)
    labelling_features[i]=["Kmer"]+datasets[i]["aMetagenomes"]+datasets[i]["aSGenomes"]+datasets[i]["mMetagenomes"]
    modelling_features[i]=["Kmer"]+[x for x in clean_names if x not in labelling_features[i]]
    labelling_matrices[i]=combined_total.drop(modelling_features[i][1:],axis=1)
    modelling_matrices[i]=combined_total.drop(labelling_features[i][1:],axis=1)
    indeces[i]["intersection"]=set(labelling_matrices[i].loc[(labelling_matrices[i][datasets[i]["aMetagenomes"]]>0)&\
                            (labelling_matrices[i][datasets[i]["mMetagenomes"]]>0)&\
                            (labelling_matrices[i][datasets[i]["aSGenomes"]]>0)]["Kmer"])
    indeces[i]["aMetagenomes_aSGenomes"]=set(labelling_matrices[i].loc[(labelling_matrices[i][datasets[i]["aMetagenomes"]]>0)&\
                            (labelling_matrices[i][datasets[i]["mMetagenomes"]]==0)&\
                            (labelling_matrices[i][datasets[i]["aSGenomes"]]>0)]["Kmer"])
    indeces[i]["aMetagenomes_mMetagenomes"]=set(labelling_matrices[i].loc[(labelling_matrices[i][datasets[i]["aMetagenomes"]]>0)&\
                            (labelling_matrices[i][datasets[i]["mMetagenomes"]]>0)&\
                            (labelling_matrices[i][datasets[i]["aSGenomes"]]==0)]["Kmer"])
    indeces[i]["mMetagenomes_aSGenomes"]=set(labelling_matrices[i].loc[(labelling_matrices[i][datasets[i]["aMetagenomes"]]==0)&\
                            (labelling_matrices[i][datasets[i]["mMetagenomes"]]>0)&\
                            (labelling_matrices[i][datasets[i]["aSGenomes"]]>0)]["Kmer"])
    indeces[i]["onlyaSGenomes"]=set(labelling_matrices[i].loc[(labelling_matrices[i][datasets[i]["aMetagenomes"]]==0)&\
                            (labelling_matrices[i][datasets[i]["mMetagenomes"]]==0)&\
                            (labelling_matrices[i][datasets[i]["aSGenomes"]]>0)]["Kmer"])
    indeces[i]["onlyaMetagenomes"]=set(labelling_matrices[i].loc[(labelling_matrices[i][datasets[i]["aMetagenomes"]]>0)&\
                            (labelling_matrices[i][datasets[i]["mMetagenomes"]]==0)&\
                            (labelling_matrices[i][datasets[i]["aSGenomes"]]==0)]["Kmer"])
    indeces[i]["onlymMetagenomes"]=set(labelling_matrices[i].loc[(labelling_matrices[i][datasets[i]["aMetagenomes"]]==0)&\
                            (labelling_matrices[i][datasets[i]["mMetagenomes"]]>0)&\
                            (labelling_matrices[i][datasets[i]["aSGenomes"]]==0)]["Kmer"])

    
    

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


In [ ]:
#Verify we don't have duplicated splittings
from itertools import combinations
for each in combinations(range(0,10),2):
    labelling_features[each[0]].sort()
    labelling_features[each[1]].sort()
    modelling_features[each[0]].sort()
    modelling_features[each[1]].sort()
    if labelling_features[each[0]]==labelling_features[each[1]]:
        print("duplicate found: "+ str(each))
    if modelling_features[each[0]]==modelling_features[each[1]]:
        print("duplicate found: "+ str(each))  

In [ ]:
classification_2.shape[0]

In [ ]:
proportions=dict.fromkeys(range(0,10))
modelling_samples=dict.fromkeys(range(0,10))
x=dict.fromkeys(range(0,10))
for i in proportions.keys():
    for j in indeces[1].keys():
        proportions[i][j]=len(indeces[i][j])/classification_2.shape[0]
n=100000
for i in range(0,10):
    to_take=[]
    labels=[]
    for j in indeces[1].keys():
        dummie=random.sample(indeces[i][j],int(proportions[i][j]*n))
        to_take=to_take+dummie
        if j in ["aMetagenomes_aSGenomes","mMetagenomes_aSGenomes","onlyaSGenomes"]:
            labels=labels+[True]*len(dummie)
        else:
            labels=labels+[False]*len(dummie)
    modelling_samples[i]=to_take
    labels_per_dataset[i]=labels

In [ ]:
from sklearn.preprocessing import Binarizer
binarized_modelling_matrices=dict.fromkeys(range(0,10))
for i in binarized_modelling_matrices.keys():  
    dummie=modelling_matrices[i][combined_total_clean["Kmer"] in modelling_samples[i]]
    dummie=dummie.drop("Kmer",axis=1).values()
    transformer = Binarizer()  # fit does nothing.
    binarized_modelling_matrices[i]=transformer.fit_transform(dummie)

In [81]:
modelling_matrices[1]

,SRR957739,SRR957740,SRR957741,SRR957744,SRR957745,SRR6877292,SRR6877372,SRR6877370,SRR6877378,SRR6877388,SRR6877382,SRR6877350,SRR6877367,SRR6877365,SRR6877361,SRR6877369,SRR6877338,SRR6877336,SRR6877334,SRR6877330,SRR6877344,SRR6877348,SRR6877340,SRR6877399,SRR6877401,SRR6877403,SRR6877328,SRR6877339,SRR6877316,SRR6877318,SRR6877322,SRR6877312,SRR6877310,SRR6877326,SRR6877393,SRR9898613,SRR5297130,SRR5297132,SRR11176627,SRR11176630,SRR11176635,SRR11176636,SRR11176638,SRR11176641,SRR12462938,SRR12462941,SRR12462942,SRR12462943,SRR12462931,SRR12462932,SRR12462934,SRR12462935,SRR12462936,SRR12462947,SRR12136713,SRR12136712,SRR5047194,SRR5047075,SRR5047132,SRR5047159,SRR5047157,SRR5047057,SRR5047086,SRR5047187,SRR5047043,SRR5047101,SRR5047146,SRR5047070,SRR5047118,SRR5047138,SRR5047077,SRR5047156,SRR5047176,SRR5047203,SRR5047126,SRR5047195,SRR5047045,SRR5047100,SRR5047106,SRR5047088,SRR5047191,SRR5047134,SRR5047094,SRR5047073,SRR5047109,SRR5047081,SRR5047140,SRR5047090,SRR5047082,SRR5047083,SRR5047113,SRR5047172,SRR5047201,SRR5047117,SRR5047169,SRR5047062,SRR5047152,SRR5047182,SRR5047071,SRR5047063,SRR5047054,SRR5047178,SRR5047123,SRR5047136,SRR5047047,SRR5047092,SRR5047199,SRR5047074,SRR5047108,SRR5047102,SRR5047179,SRR5047144,SRR5047121,SRR5047098,SRR5047124,SRR5047087,SRR5047184,SRR5047066,SRR5047093,SRR5047167,SRR5047173,SRR5047107,SRR5047059,SRR5047163,SRR5047049,SRR5047145,SRR5047061,SRR5047149,SRR5047112,SRR5047154,SRR5047204,SRR5047153,SRR5047056,SRR5047130,SRR5047139,SRR5047131,SRR5047072,SRR5047150,SRR5047084,SRR5047105,SRR5047064,ERR1883904,ERR2204628,SRR12548766,SRR1794618,SRR1783758,SRR1795018,SRR5892215,SRR769525,SRR769515,SRR769527,SRR769519
npartitions=6906,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,.